# Imports

In [10]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

# Custom Environmnet

In [11]:
class Shower(Env):
    def __init__ (self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low = np.array([1]), high = np.array([100]))
        self.state = 38 + random.randint(-3, 3)
        self.shower_lenght = 60
    def step(self, action):
        self.state += action - 1
        self.shower_lenght -= 1

        #reward calculation
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_lenght == 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info
    def render(self):
        pass
    def reset(self):
        self.state = 38 + random.randint(-3, 3)
        self.shower_lenght = 60
        return self.state


In [12]:
env = Shower()

C:\Python310\lib\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [13]:
env.action_space.sample()

0

In [14]:
env.observation_space.sample()

array([9.172595], dtype=float32)

# Model Testing without training

In [15]:
episode = 20
for i in range(1, episode+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {i} Score: {score}")

Episode: 1 Score: -48
Episode: 2 Score: -38
Episode: 3 Score: -6
Episode: 4 Score: 6
Episode: 5 Score: -8
Episode: 6 Score: -60
Episode: 7 Score: -42
Episode: 8 Score: -50
Episode: 9 Score: -12
Episode: 10 Score: -58
Episode: 11 Score: -2
Episode: 12 Score: -30
Episode: 13 Score: -30
Episode: 14 Score: -60
Episode: 15 Score: -40
Episode: 16 Score: -42
Episode: 17 Score: -30
Episode: 18 Score: -34
Episode: 19 Score: -10
Episode: 20 Score: 8


# Deep Learning Model

In [16]:
from keras import layers
from keras import models
import tensorflow as tf

In [17]:
def build_model(states, actions):
    network = models.Sequential()
    network.add(layers.Flatten(input_shape = states))
    network.add(layers.Dense(24, activation='relu', input_shape = (1,))) # x1 is our input
    network.add(layers.Dense(24, activation='relu'))
    network.add(layers.Dense(actions, activation='linear')) # output layer
    return network

In [23]:
del model

In [24]:
states = env.observation_space.shape
actions = env.action_space.n

In [25]:
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# Training Agent using RL

In [26]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from tensorflow.keras.optimizers import Adam

In [27]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length = 1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
        nb_actions=actions, nb_steps_warmup=10, target_model_update=1)
    return dqn

In [28]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

C:\Python310\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 10:17 - reward: -1.0000

C:\Python310\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Python310\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
C:\Python310\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   33/10000 [..............................] - ETA: 3:10 - reward: -1.0000

C:\Python310\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Python310\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Python310\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Python310\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Python310\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs = np.random.random_integers(low, high -

10000/10000 [==============================] - 83s 8ms/step - reward: -0.8216
166 episodes - episode_reward: -49.410 [-60.000, 10.000] - loss: 0.961 - mae: 11.025 - mean_q: -15.593

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: -0.5816
167 episodes - episode_reward: -34.790 [-60.000, 34.000] - loss: 1.561 - mae: 11.362 - mean_q: -16.233

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: -0.4826 0s - rewar
167 episodes - episode_reward: -29.018 [-60.000, 44.000] - loss: 1.518 - mae: 9.776 - mean_q: -13.850

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 90s 9ms/step - reward: 0.0242
166 episodes - episode_reward: 1.349 [-46.000, 46.000] - loss: 0.671 - mae: 3.929 - mean_q: -4.127

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 81s 8ms/step - reward: 0.0424
done, took 416.551 seconds
